### Scraping current supercharger data from Teslas website

1.	Import the necessary modules - requests, BeautifulSoup, pandas, and re.
2.	Set the variable "url" to the URL of a webpage containing a list of Tesla supercharger locations in the United States.
3.	Use the requests module to send a GET request to the URL and store the response in the "response" variable.
4.	Use BeautifulSoup to parse the HTML content of the response and store the result in the "soup" variable.
5.	Find all the "span" elements with the "adr" class in the "soup" object and extract the street address and locality information from each element. Store the results in a list of dictionaries called "locations".
6.	Define a function called "remove_punctuation" to remove all punctuation characters from specified columns of a Pandas DataFrame.
7.	Create a Pandas DataFrame called "df" from the "locations" list of dictionaries.
8.	Replace newline characters in the "locality" column of the DataFrame with a space character.
9.	Apply the "remove_punctuation" function to the "street_address" and "locality" columns of the DataFrame.
10.	Combine the "street_address" and "locality" columns of the DataFrame into a new column called "full_address".
11.	Import the Google Maps API client library and authenticate using a private key.
12.	Define a function called "get_lat_lng" that takes an address string as input, sends a geocoding request to the Google Maps API, and returns the latitude and longitude of the address (if available).
13.	Apply the "get_lat_lng" function to the "full_address" column of the DataFrame using the "apply" method and store the results in new "latitude" and "longitude" columns.


In [ ]:
#Google maps private key
private_key ='YOUR KEY HERE'

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

url = 'https://www.tesla.com/findus/list/superchargers/United%20States'

response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

locations = soup.find_all('span', class_='adr')

locations = []
for span in soup.find_all('span', {'class': 'adr'}):
    location = {}
    location['street_address'] = span.find('span', {'class': 'street-address'}).text.strip()
    location['locality'] = span.find('span', {'class': 'locality'}).text.strip()
    locations.append(location)
    
    
def remove_punctuation(df, cols):
    for col in cols:
        df[col] = df[col].apply(lambda x: re.sub('[^\w\s]','',x))
    return df
    
df = pd.DataFrame(locations)
df['locality'] = df['locality'].str.replace('\n', ' ')
df = remove_punctuation(df, ['street_address', 'locality'])
df['full_address'] = df['street_address'].str.replace('\W', ' ') + ' ' + df['locality'].str.replace('\W', ' ')
df

In [33]:
import googlemaps
gmaps = googlemaps.Client(key=private_key)

def get_lat_lng(address):
    geocode_result = gmaps.geocode(address)
    if len(geocode_result) > 0:
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        return lat, lng
    else:
        return None, None



In [34]:
df['latitude'], df['longitude'] = zip(*df.apply(lambda x: get_lat_lng(f"{x['street_address']}, {x['locality']}"), axis=1))


In [35]:
df

,street_address,locality,full_address,latitude,longitude
0,21282 AthensLimestone Blvd,Athens AL ...,21282 AthensLimestone Blvd...,34.785789,-86.942933
1,1627 Opelika Road,Auburn AL ...,1627 Opelika Road Auburn ...,32.627776,-85.448195
2,1617 South College Street,Auburn AL ...,1617 South College Street...,32.576356,-85.496440
3,2221 Richard Arrington Jun...,Birmingham AL ...,2221 Richard Arrington Jun...,33.534810,-86.813383
4,6350 Interstate Drive,Cottondale AL ...,6350 Interstate Drive Cot...,33.174913,-87.450532
...,...,...,...,...,...
1597,730 S Main St,Lusk WY ...,730 S Main St Lusk ...,42.755782,-104.452633
1598,2370 E Cedar St,Rawlins WY ...,2370 E Cedar St Rawlins ...,41.791310,-107.210632
1599,2441 Foothill Blvd,Rock Springs WY ...,2441 Foothill Blvd Rock Sp...,41.581100,-109.263894
1600,612 North Main Street,Sheridan WY ...,612 North Main Street Sher...,44.804096,-106.956266
